## Segmenting and Clustering Neighborhoods in Toronto- Part 1



**Pre-Processing**

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Collect data from Wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Get the HTML text from the web page using requests module.


In [4]:
# load html text from the wikipedia url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_txt = requests.get(url).text

# Using BeautifulSoup Module extract text
soup = BeautifulSoup(html_txt)
table = soup.find('table', attrs={'class':'wikitable sortable'})
places = table.find_all('tr')

# Extract text from table cells and add rows to extraced list of rows
rows = list()
for tr in places:
    post = tr.find_all('td')
    row = [ele.text.strip() for ele in post]
    if row:
        rows.append(row)

print("Number of rows with data in the table: ", len(rows))

Number of rows with data in the table:  180


**Create Dataframe Using Pandas for Neighborhood Data. Remove "Not Assigned Columns**

In [5]:
df = pd.DataFrame(rows,columns=['PostalCode', 'Borough', 'Neighborhood'])

df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)
print("Dataframe shape: ", df.shape)
df.head(10)

Dataframe shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


**Replace 'Neighborhood' columns with value as 'Not assigned' with the value of its 'Borough'**

In [6]:

df['Neighborhood'] = df.apply(
    lambda row: 
    row['Borough'] if row['Neighborhood'] == 'Not assigned' 
    else row['Neighborhood'],
    axis=1)
print("Dataframe shape: ", df.shape)
df.head(10)

Dataframe shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


**Combine rows with same postal code into single row**

In [7]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)
df.head(15)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df.shape

(103, 3)